In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

cars = np.genfromtxt("cars.csv", delimiter=",", skip_header=True)

cars_columns_min = np.min(cars, axis=0)
cars_columns_max = np.max(cars, axis=0)
cars_normalized = (cars - cars_columns_min) / (cars_columns_max - cars_columns_min)

mpg = cars_normalized[:, 7]

In [ ]:
#TODO 1: create weighted function with coefficient for each column parameter

def weighted_function (x, coefficients):
    return np.dot(x, coefficients)

def coeffients(number):
    cV = np.random.uniform(low= -1.0, high=1.0, size=number)
    return cV

def offspring(parent):
    return parent + np.random.uniform(low= -0.1, high=0.1, size=parent.shape)


def fitness_function (data, coefficients):
    return mean_squared_error(data, weighted_function(data, coefficients))

